In [1]:
from utils.spark_config import get_spark_session
from configs.paths import ROOT_DIR

spark = get_spark_session()

:: loading settings :: url = jar:file:/home/administrator/Desktop/datascience/github/learning-spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/administrator/.ivy2.5.2/cache
The jars for the packages stored in: /home/administrator/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df136be1-9d27-42a6-9cb6-f7216ba5833d;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 162ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|             

In [2]:
churn_df = spark.read.format("delta").load(str(ROOT_DIR / "data" / "silver" / "customer_churn_silver"))
churn_df.show(5)

25/08/20 19:08:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------------+-----+
|customerID|            features|label|
+----------+--------------------+-----+
|7590-VHVEG|(26,[1,3,6,11,13,...|  0.0|
|5575-GNVDE|(26,[1,5,7,13,14,...|  0.0|
|3668-QPYBK|(26,[1,3,7,13,14,...|  1.0|
|7795-CFOCW|(26,[1,5,9,13,16,...|  0.0|
|9237-HQITU|(26,[0,3,6,13,14,...|  1.0|
+----------+--------------------+-----+
only showing top 5 rows


In [3]:
attrs = churn_df.schema["features"].metadata["ml_attr"]["attrs"]

# Flatten metadata dict into feature names
feature_names = []
for attr_type in ["binary", "numeric", "nominal"]:
    if attr_type in attrs:
        feature_names += [x["name"] for x in attrs[attr_type]]

print(feature_names)

['InternetService_index_ohe_Fiber optic', 'InternetService_index_ohe_DSL', 'InternetService_index_ohe_No', 'Contract_index_ohe_Month-to-month', 'Contract_index_ohe_Two year', 'Contract_index_ohe_One year', 'PaymentMethod_index_ohe_Electronic check', 'PaymentMethod_index_ohe_Mailed check', 'PaymentMethod_index_ohe_Credit card (automatic)', 'PaymentMethod_index_ohe_Bank transfer (automatic)', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn']


In [4]:
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

# Convert features vector to array
churn = churn_df.withColumn("features_array", vector_to_array(col("features")))

churn.show(5, truncate=False)

+----------+---------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------+
|customerID|features                                                                               |label|features_array                                                                                                                          |
+----------+---------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------+
|7590-VHVEG|(26,[1,3,6,11,13,17,22,23,24],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,29.85,29.85])               |0.0  |[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 29.85, 29.85, 0.0]  |
|5575-GNVDE|(26,[1,5,7,1

In [5]:
for column_name in feature_names:
    churn = churn.withColumn(column_name, col("features_array")[feature_names.index(column_name)])


churn.show(5, truncate=False)

+----------+---------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+-----------------------------------------------+-------------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+-----+
|customerID|features                                                                               |label|features_array                                                                      

In [6]:
primary_key = "CustomerID"
target_column = "Churn"
feature_names = [name for name in feature_names if name not in [primary_key, target_column]]

churn = churn.select(primary_key, *feature_names, target_column)

churn.show(5)

+----------+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+-----------------------------------------------+-------------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+-----+
|CustomerID|InternetService_index_ohe_Fiber optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month-to-month|Contract_index_ohe_Two year|Contract_index_ohe_One year|PaymentMethod_index_ohe_Electronic check|PaymentMethod_index_ohe_Mailed check|PaymentMethod_index_ohe_Credit card (automatic)|PaymentMethod_index_ohe_Bank transfer (automatic)|SeniorCitizen|Partner|Dependents|tenure|PhoneS

In [7]:
import re

# Function to clean names
def clean_colname(name):
    name = re.sub(r"[()]", "", name)   # remove parentheses
    name = re.sub(r"[\s\-]+", "_", name)  # replace spaces/hyphens with underscore
    return name

# Apply to all columns
cleaned_cols_churn = [clean_colname(c) for c in churn.columns]

churn = churn.toDF(*cleaned_cols_churn)

In [8]:
primary_key = "CustomerID"
target_column = "Churn"
feature_names = [name for name in churn.columns if name not in [primary_key, target_column]]

churn = churn.select(primary_key, *feature_names, target_column)
churn.show(5)

+----------+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+---------------------------------------------+-----------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+-----+
|CustomerID|InternetService_index_ohe_Fiber_optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month_to_month|Contract_index_ohe_Two_year|Contract_index_ohe_One_year|PaymentMethod_index_ohe_Electronic_check|PaymentMethod_index_ohe_Mailed_check|PaymentMethod_index_ohe_Credit_card_automatic|PaymentMethod_index_ohe_Bank_transfer_automatic|SeniorCitizen|Partner|Dependents|tenure|PhoneService|M

In [9]:
churn.select(primary_key, *feature_names).write.format("delta").mode("overwrite").save(str(ROOT_DIR / "data" / "ml" / "customer_churn_features"))

In [10]:
spark.stop()
